In [23]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.py'>

In [24]:
ntuples = '20170509'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
signal = [s for s in files if "NonRes" in s]
diphotonJets = [s for s in files if "DiPhotonJets" in s]
gJets = [s for s in files if "GJet" in s]

utils.IO.add_signal(ntuples,signal)
utils.IO.add_background(ntuples,diphotonJets)
utils.IO.add_background(ntuples,gJets)
for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/output_GJet_total.root
using signal file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/AllNonResSig.root


In [ ]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
branch_names = 'leadingJet_bDis,subleadingJet_bDis,CosThetaStar_CS,CosTheta_bb,CosTheta_gg,noexpand:diphotonCandidate.Pt(),noexpand:dijetCandidate.Pt(),noexpand:diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
branch_names = (b.replace(" ", "_") for b in branch_names)
branch_names = list(b.replace("-", "_") for b in branch_names)
print branch_names

import pandas as pd
import root_pandas as rpd

signal_df = []
background_df = []

In [43]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
branch_names = 'leadingJet_bDis,subleadingJet_bDis,CosThetaStar_CS,CosTheta_bb,CosTheta_gg,noexpand:diphotonCandidate.Pt(),noexpand:dijetCandidate.Pt(),noexpand:diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
branch_names = (b.replace(" ", "_") for b in branch_names)
branch_names = list(b.replace("-", "_") for b in branch_names)
print branch_names

import pandas as pd
import root_pandas as rpd

signal_df = []
background_df = []
weights_sig = []
weights_bkg = []

weights_sig.append(1)
weights_bkg.append(0.5)
weights_bkg.append(0.5)

shuffle = True



for i in range(utils.IO.nSig):
    signal_df.append(rpd.read_root(utils.IO.signalName[i],"bbggSelectionTree", columns = branch_names))
    if shuffle:
        signal_df[i]['random_index'] = np.random.permutation(range(signal_df[i].index.size))
        signal_df[i].sort_values(by='random_index',inplace=True)

    utils.preprocessing.adjust_and_compress(signal_df[i]).to_hdf('/tmp/micheli/signal.hd5','sig',compression=9,complib='bzip2',mode='a')
    utils.preprocessing.define_process_weight(signal_df[i],i+1,weights_sig[i])

for i in range(utils.IO.nBkg):
    background_df.append(rpd.read_root(utils.IO.backgroundName[i],"bbggSelectionTree", columns = branch_names))
    utils.preprocessing.adjust_and_compress(background_df[i]).to_hdf('/tmp/micheli/background.hd5','bkg',compression=9,complib='bzip2',mode='a')
    utils.preprocessing.define_process_weight(background_df[i],-(i+1),weights_bkg[i])
    
y_bkg = [x[['proc']] for x in background_df]
print y_bkg

['leadingJet_bDis', 'subleadingJet_bDis', 'CosThetaStar_CS', 'CosTheta_bb', 'CosTheta_gg', 'noexpand:diphotonCandidate.Pt()', 'noexpand:dijetCandidate.Pt()', 'noexpand:diHiggsCandidate.M()']
[         proc
0          -1
1          -1
2          -1
3          -1
4          -1
5          -1
6          -1
7          -1
8          -1
9          -1
10         -1
11         -1
12         -1
13         -1
14         -1
15         -1
16         -1
17         -1
18         -1
19         -1
20         -1
21         -1
22         -1
23         -1
24         -1
25         -1
26         -1
27         -1
28         -1
29         -1
...       ...
2443042    -1
2443043    -1
2443044    -1
2443045    -1
2443046    -1
2443047    -1
2443048    -1
2443049    -1
2443050    -1
2443051    -1
2443052    -1
2443053    -1
2443054    -1
2443055    -1
2443056    -1
2443057    -1
2443058    -1
2443059    -1
2443060    -1
2443061    -1
2443062    -1
2443063    -1
2443064    -1
2443065    -1
2443066    -1
2443067   